# Solvent selection using Chem_VAE-generated molecular descriptors - with UDM for filehandling

Workflow:
1. Create 2 pandas df from my 2 UDM files. One with the CAS and SMILES of all solvents used, and another with the CAS, d.e. and conv. Use these two UDM files to create two pandas dfs.
2. Use Chem_VAE to generate an array of molecular discriptors in the latent space for each solvent (1x196)
3. 196 discriptors is too many to train a GP model on - do PCA for dimensionality reduction to arrive at fewer discriptors
4. Then a random_forest model with the descriptors as x and yields as y is used to predict the optimal next solvent to test
5. Finally, a UDM file with the CAS and SMILES of this solvent is produced

# 1. Import the functions needed

In [24]:
#Normal imports required
import pandas as pd
import numpy as np
from numpy import shape
import xml.etree.ElementTree  as ET
from lxml.builder import ElementMaker # lxml only !
from lxml import etree
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import scikitplot as skplt
import datetime
from lxml import etree
from io import StringIO
from urllib.request import urlopen
E = ElementMaker()

#My functions defined in UDM_converters.py
from UDM_converters import udm_to_df
from UDM_converters import CAS_to_SMILES
from UDM_converters import prediction_udm_file
from UDM_converters import UDM_verifier

#Imports required for the VAE
#from https://github.com/aspuru-guzik-group/chemical_vae

# tensorflow backend
from os import environ
environ['KERAS_BACKEND'] = 'tensorflow'
# vae stuff
import sys
sys.path.insert(1, '/Users/dsw46/OneDrive - University of Cambridge/Cambridge/\
udm_project/Solvent_selection/chemical_vae_master')
from chemvae.vae_utils import VAEUtils
from chemvae import mol_utils as mu
# import scientific py
import numpy as np
import pandas as pd
# rdkit stuff
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import PandasTools
# plotting stuff
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import SVG, display
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

#load the vae model
vae = VAEUtils(directory='chemical_vae_master/models/zinc_properties')

#ML imports
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process import GaussianProcessRegressor

/Users/dsw46/opt/anaconda3/envs/chemvae/lib/python3.6/site-packages/keras/models.py:245: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Using standarized functions? True
Standarization: estimating mu and std values ...done!


ImportError: cannot import name 'GaussianProcessRegressor'

# 2. Define the functions needed

In [4]:
def SMILES_to_VAE(dataframe):
    """
    This function takes in a dataframe of molecules (e.g. solvents) that have smiles strings under the heading
    "MOLSTRUCTURE" and creates an array with the VAE generated molecular descriptors.
    """
    VAE_array = np.zeros([len(dataframe['MOLSTRUCTURE']), 196])
    for i in range(len(dataframe['MOLSTRUCTURE'])):
        try:
            smiles_1=mu.canon_smiles(dataframe['MOLSTRUCTURE'][i])
            X_1 = vae.smiles_to_hot(smiles_1,canonize_smiles=True)
            z_1 = vae.encode(X_1)
            VAE_array[i, :] = z_1
        except:
            VAE_array[i, :] = 0
            print("Converting molecule in index ", i, "didn't work")
    return(VAE_array)    

There are 4 unconvertable "bad SMILES" in the dataset

In [5]:
def my_PCA(array, num_components):
    '''
    PCA takes in an array of size (x,y) with x components, with each component being described by y dimensions.
    In this notebook each component is a solvent, and the y-values are generated by the Chem_VAE.
    num_components is the number of components you want in your output PCA.
    The function returns an array of size (x, z), where z=num_components
    Code copied from Kobi's summit
    '''
    pca = PCA(n_components=num_components)
    pcs_practical = pca.fit_transform(array)
    print(f"{round(pca.explained_variance_ratio_.sum()*100)}% of variance\
    is explained by {num_components} principal components")
    return(pcs_practical)

In [11]:
#This block will get you a graph showing the variance explained as a funciton of PCA dimensions

# pca = PCA(random_state=1)
# pca.fit(VAE_list)
# skplt.decomposition.plot_pca_component_variance(pca)
# plt.show()

Above plot shows how much variance is explained depending on the number of principle components.

### Now I can move on to training my surrogate model: A random forest

Next step is to link the cas numbers, such that the latent vectors are linked to the correct conversions/d.e.

I need to make sure the input dataset (the latent vectors, as 'X') is linked to the corresponding output data (conversion as 'y'). 
NB: Since solvent choice only has very little influence on the d.e., and the number of solvent experiments is relatively low, we will only use conversion in the model.

In [12]:
def CAS_matching(results_df, all_solvents_df, PCA_all_solvents):
    """
    This function returns an array of Chem VAE mol descriptors reduced by PCA, such that the entries are in the same
    order as for the results dataframe (by matching CAS numbers).
    
    - results_df: The pandas df containing results, i.e. conversion and d.e. Must have 'CAS' as a column.
    - all_solvents_df: The pandas df containing all solvents (with CAS numbers). Must have 'CAS' as a column.
    - PCA_all_solvents: An array containing the result of applying PCA to the VAE mol descriptors.
    """
    X_temp=np.array([PCA_all_solvents[0]])
    for i in range(len(results_df)):
        for j in range(len(all_solvents_df)):
            if results_df['CAS'][i] == all_solvents_df['CAS'][j]:
                a = np.array([PCA_all_solvents[j]])
                X_temp = np.concatenate([X_temp,a])
    X_temp=np.delete(X_temp,0,0)
    X=X_temp
    return(X)

## Now that we finally have the data in the right shape, let's try implementing random forest regressor

In [34]:
def surrogate_model(X, y, PCA_all_solvents, all_solvents_df, model_choice):
    """
    This function implements the random_forest_regressor to predict the optimal solvent for the experiment.
    - X: PCA reduced descriptors for each solvent used in the experiment, in same order as y
    - y: conversion for each solvent used in the experiment
    - PCA_all_solvents: array with the PCA reduced mol descriptors of all solvents
    - conv_max: Is a list with 2 numbers (Predicted max conversion achievable, index of this solvent)
    - best_solvent: CAS number of the predicted best solvent
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
    
    if model_choice == 'random_forest_regressor':
        regr = RandomForestRegressor(criterion= 'mae', random_state = 0)
    elif model_choice == 'linear_regressor':
        regr = LinearRegression()
    elif model_choice == 'gaussian_process_regressor':
        regr = GaussianProcessRegressor(random_state = 0)
    else:
        print('That is not a valid surrogate model, please try again')
        quit()
    
    # fit the model
    regr.fit(X_train, y_train)

    # we can now use it like any other estimator
    print('R^2 error on the training data (best is 1.0) is: ', r2_score(y_train, regr.predict(X_train)))
    print('R^2 error on the test data (best is 1.0) is: ', r2_score(y_test, regr.predict(X_test)))
    
    # predicting the best solvent for conversion using the mol descriptors reduced by PCA
    predictions = regr.predict(PCA_all_solvents)
    conv_max = max_of_array(predictions)
    print('Predicted max achievable conversion is ', conv_max[0])
    best_solvent = all_solvents_df['CAS'][conv_max[1]]
    return(best_solvent)

#### Assuming I'd built a model with good predictive power, I'd now like to take the whole solvent dataset into account, and try to predict the best solvent.
NB: Since we're just training the model to predict conversion (1D) and not "conv. and d.e." (2D), we don't need a pareto front

In [15]:
def max_of_array(array):
    overall_max = 0
    for i in range(len(array)):
        temp_max = array[i]
        if temp_max > overall_max:
            overall_max = temp_max
            overall_max_index = i
    return(overall_max, overall_max_index)

# Now create a UDM file containing just that cas number

# 3. Define the solvent predictor function

In [16]:
def Solvent_predictor(UDM_schema, solvent_collection, solvent_results, num_PCA_components, output_UDM_file, model_choice):
    """
    This function outputs a UDM file with the predicted best solvent to experiment with next.
    
    It takes the following inputs:
    - solvent_collection: A UDM file containing all solvents to be considered in the experiment. It has colums 
        with the molecular structure (column name: 'MOLSTRUCTURE', a SMILES string), molecular weight 
        (name: 'MW'), name (name: 'NAME'), and CAS number (name: 'CAS'). Other properties can also be added in 
        other columns.
    - solvent results: A UDM file containing the solvents used in an experiment which has the same columns as
        "solvent_collection", and also the columns "conv" and "d.e." for conversion and diastereomeric excess,
        respectively.
    - num_PCA_components: An integer. The number of principle components in the PCA.
    - model_choice: Can either be 'random_forest_regressor', 'gaussian_progress_regressor', or 'linear_regressor'
    """
    #Verify the UDM files
    UDM_verifier(solvent_collection, UDM_schema)
    UDM_verifier(solvent_results, UDM_schema)
    
    #Import the UDM files
    full_all_solvents_df=udm_to_df(solvent_collection)
    full_results_df=udm_to_df(solvent_results)

    #Extract only the CAS numbers and SMILES strings from the all_solvents_df
    #in this instance, the MOLSTRUCTURE is a SMILES string
    all_solvents_df = full_all_solvents_df[['CAS', 'MOLSTRUCTURE']]

    #Extract only the CAS numbers, conversions and d.e. from the full_results_df
    results_df = full_results_df[['CAS', 'conv', 'd.e.']]
    
    
    #Create an array with the VAE mol descriptors for each solvent
    VAE_array = SMILES_to_VAE(all_solvents_df)
    
    #Do PCA on the VAE array
    PCA_array = my_PCA(VAE_array, num_PCA_components)
    
    #Define X and y for the ML model
    y = results_df['conv'].copy()
    X = CAS_matching(results_df, all_solvents_df, PCA_array)
    
    # Use the model to predict the best solvent
    CAS_best_solvent = surrogate_model(X, y, PCA_array, all_solvents_df, model_choice)
    
    # Create a UDM file with the result
    prediction_udm_file(CAS_best_solvent, output_UDM_file)
    


In [22]:
#Linear regression
Solvent_predictor('udm_6_0_0.xsd', 'all_solvents.xml', 'solvents_results.xml', 11, 'predicted_best_solvent.xml', 'linear_regressor')

XML well formed, syntax ok.
XML valid, schema validation ok.
XML well formed, syntax ok.
XML valid, schema validation ok.


RDKit WARNING: [16:57:45] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [16:57:45] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [16:57:45] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [16:57:45] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [16:57:45] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [16:57:45] WARNING: not removing hydrogen atom without neighbors


ERROR: Check chars file. Bad SMILES: CN1CCCCC1.[Cl-].[H+]                                                                                                    
Converting molecule in index  345 didn't work
ERROR: Check chars file. Bad SMILES: NNc1ccccc1.[Cl-].[H+]                                                                                                   
Converting molecule in index  372 didn't work
ERROR: Check chars file. Bad SMILES: C1CCNC1.[Cl-].[H+]                                                                                                      
Converting molecule in index  393 didn't work
ERROR: Check chars file. Bad SMILES: C[Si](C)(C)C                                                                                                            
Converting molecule in index  402 didn't work
77.0% of variance    is explained by 11 principal components
R^2 error on the training data (best is 1.0) is:  0.5841151903357276
R^2 error on the test data (best is 1.0) is:  -0.53403

In [27]:
#gaussian process
Solvent_predictor('udm_6_0_0.xsd', 'all_solvents.xml', 'solvents_results.xml', 11, 'predicted_best_solvent.xml', 'gaussian_process_regressor')

XML well formed, syntax ok.
XML valid, schema validation ok.
XML well formed, syntax ok.
XML valid, schema validation ok.


RDKit WARNING: [17:12:51] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:12:51] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:12:51] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:12:51] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:12:51] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:12:51] WARNING: not removing hydrogen atom without neighbors


ERROR: Check chars file. Bad SMILES: CN1CCCCC1.[Cl-].[H+]                                                                                                    
Converting molecule in index  345 didn't work
ERROR: Check chars file. Bad SMILES: NNc1ccccc1.[Cl-].[H+]                                                                                                   
Converting molecule in index  372 didn't work
ERROR: Check chars file. Bad SMILES: C1CCNC1.[Cl-].[H+]                                                                                                      
Converting molecule in index  393 didn't work
ERROR: Check chars file. Bad SMILES: C[Si](C)(C)C                                                                                                            
Converting molecule in index  402 didn't work
77.0% of variance    is explained by 11 principal components
R^2 error on the training data (best is 1.0) is:  1.0
R^2 error on the test data (best is 1.0) is:  -6.858357201646722
Pred

In [33]:
#random forest regressor
Solvent_predictor('udm_6_0_0.xsd', 'all_solvents.xml', 'solvents_results.xml', 11, 'predicted_best_solvent.xml', 'random_forest_regressor')

XML well formed, syntax ok.
XML valid, schema validation ok.
XML well formed, syntax ok.
XML valid, schema validation ok.


RDKit WARNING: [18:16:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:16:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:16:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:16:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:16:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:16:44] WARNING: not removing hydrogen atom without neighbors


ERROR: Check chars file. Bad SMILES: CN1CCCCC1.[Cl-].[H+]                                                                                                    
Converting molecule in index  345 didn't work
ERROR: Check chars file. Bad SMILES: NNc1ccccc1.[Cl-].[H+]                                                                                                   
Converting molecule in index  372 didn't work
ERROR: Check chars file. Bad SMILES: C1CCNC1.[Cl-].[H+]                                                                                                      
Converting molecule in index  393 didn't work
ERROR: Check chars file. Bad SMILES: C[Si](C)(C)C                                                                                                            
Converting molecule in index  402 didn't work
77.0% of variance    is explained by 11 principal components
R^2 error on the training data (best is 1.0) is:  0.91593467711916
R^2 error on the test data (best is 1.0) is:  -0.6153935